In [ ]:
from google.cloud import storage
import os
from io import StringIO # if going with no saving csv file
import gcsfs
import dask.dataframe as dd
from dask.delayed import delayed
from pathlib import Path
import pandas as pd
import os

In [ ]:
data_dir = Path(os.getenv('APP_DIR') + '/nanohub_metrics/userlogin')

In [ ]:
%%timeit

full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet.gzip')
)

In [ ]:
print(full_df.head())

In [ ]:
%%timeit


filenames = data_dir.glob('*.parquet.gzip')
dfs = [delayed(pd.read_parquet)(fn) for fn in filenames]

dask_df = dd.from_delayed(dfs) 

In [ ]:
print(dask_df.head())
df = dask_df.compute()

In [ ]:
%%timeit

from multiprocessing import Pool

def reader(filename):
    return pd.read_parquet(filename)

def main():
    pool = Pool(16) # number of cores you want to use
    file_list = data_dir.glob('*.parquet.gzip')
    df_list = pool.map(reader, file_list) #creates a list of the loaded df's
    return pd.concat(df_list) # concatenates all the df's into a single df

df = main()

In [ ]:
print(df.head())
print(df.tail())